# 產生 Trainning 的 DataSet
---

In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm

In [2]:
# 這邊是要把要做的 List 塞進去
# 再來是設定 Output 檔案 
# !! (一定要有反斜線做結尾) !!
# 原本資料       => 日期 牙齒
# 原 Label 資料  => 日期 牙齒_Label
# Trainning資料  => 日期 牙齒_Trainning
DoOrgList =[
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor/",
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor2/",
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor3/",
]
PicMinNumber = 60
PicMaxNumber = 200
CutIndex = 500


# 人工 Label 的 & 最後 Trainning 要用的
LabelPath = "_Label"
InputPath = "_Input"
TrainningPath = "_Trainning"

# 這邊是 SegNet 相關的設定
TrainTxtLocation = "D:/Dentist/Data/ScanData/2018.10.18/train.txt"

WindowsDataLocation = "D:/Dentist/Data/ScanData/"
UbuntuDataLocation = "/home/cgal/Desktop/DentistData/"

In [3]:
# 把資料夾的名稱 Append 某個 String
def FileNameAppend(FileName, Type):
    return FileName[:-1] + Type + "/"

In [4]:
# 後面會用到的變數
Org_FileNameList = []
Label_FileNameList = []
Input_FileNameList = []
Trainning_FileNameList= []
OrgImageList = []
# TrainImageList = []

def CheckIsFileMissing():
    # Check 有沒有漏轉檔案 & 加進要處理的 List 中
    for i in range(len(DoOrgList)):
        for picIndex in range(PicMinNumber, PicMaxNumber + 1):
            # 路徑
            OrgFileName = DoOrgList[i] + str(picIndex) + ".png"
            LabelFileName = FileNameAppend(DoOrgList[i], LabelPath) + str(picIndex) + ".png"
            InputFileName = FileNameAppend(DoOrgList[i], InputPath) + str(picIndex) + ".png"
            TrainningFileName = FileNameAppend(DoOrgList[i], TrainningPath) + str(picIndex) + ".png"
            imgOrg  = cv2.imread(OrgFileName, cv2.IMREAD_GRAYSCALE)

            if(imgOrg is None):
                print(FileName)
                return True
            
            # 加到 List 中
            Org_FileNameList.append(OrgFileName)
            Label_FileNameList.append(LabelFileName)
            Input_FileNameList.append(InputFileName)
            Trainning_FileNameList.append(TrainningFileName)
            OrgImageList.append(imgOrg)
    return False

In [5]:
print("有漏失" if CheckIsFileMissing() else "沒有漏失")

沒有漏失


In [6]:
len(OrgImageList)

423

In [7]:
row, col = OrgImageList[0].shape
print(row)
print(col)
print(OrgImageList[0].dtype)

250
1024
uint8


## 資料轉換
---
1. 要先把資料 Resize
2. 圖片中白的 => 0
3. 圖片中黑的 => 1

In [8]:
# 假設 Trainning 的資料夾沒有建好
# 會重建一個
for i in range(len(DoOrgList)):
    if(not os.path.exists(FileNameAppend(DoOrgList[i], TrainningPath))):
        os.mkdir(FileNameAppend(DoOrgList[i], TrainningPath))
    
    if(not os.path.exists(FileNameAppend(DoOrgList[i], InputPath))):
        os.mkdir(FileNameAppend(DoOrgList[i], InputPath))

In [9]:
# 開始 Resize & 轉換資料
# SmallImageList = []
for i in tqdm(range(len(OrgImageList))):
    # Resize
    # img = cv2.resize(ImageList[i], (row / 4, col / 4), interpolation=cv2.INTER_NEAREST)
    # print(img)
    # print(img.shape)
    # break
    
    # 改圖片
    SmallOrgImage = np.zeros((row, min(col, CutIndex)), dtype=np.uint8)
    SmallImage = np.zeros((row, min(col, CutIndex)), dtype=np.uint8)
    for r in range(row):
        for c in range(min(col, CutIndex)):
            # 原圖直接丟回去
            SmallOrgImage[r][c] = OrgImageList[i][r][c]
            
            # 黑
            if(OrgImageList[i][r][c] == 0):
                SmallImage[r][c] = 1
                
            # 白
            elif(OrgImageList[i][r][c] == 255):
                SmallImage[r][c] = 0
                
    # 丟進 Stack
    # SmallImageList.append(SmallImage)
        
    # 存出目錄
    SaveLocation = Input_FileNameList[i]
    cv2.imwrite(SaveLocation, SmallOrgImage)
    SaveLocation = Trainning_FileNameList[i]
    cv2.imwrite(SaveLocation, SmallImage)

100%|████████████████████████████████████████████████████████████████████████████████| 423/423 [03:21<00:00,  1.92it/s]


## 寫 Train.txt
---
InputImage LabelImage 

In [10]:
# 檔案名字從 Windows 轉道 Ubuntu
def FileNameWindowsToUbuntu(FileName):
    return FileName.replace(WindowsDataLocation, UbuntuDataLocation)

In [11]:
TrainScript = ""
for i in range(len(Label_FileNameList)):
    TrainScript += FileNameWindowsToUbuntu(Input_FileNameList[i]) + " " + \
                        FileNameWindowsToUbuntu(Trainning_FileNameList[i]) + "\n"

In [12]:
file = open(TrainTxtLocation, "w")
file.write(TrainScript)
file.close()

## 開始丟到 Ubuntu 上 Trainning 囉
---
1. 記得只要丟 原圖 & Trainning的
2. 還有 Train.txt